In [2]:
from get_data import get_data
from blocks import init_blocks, calc_factors, get_block_stat, get_block_penalty
from students import init_students, calc_weights, get_students_penalty, update_students
from students import get_p0_students, get_students_stat, get_classes_for_module, get_incomplete_students
from planner import get_try, list_results

In [3]:
# read module data
blocks = get_data('data/blocks.json')

# read student priorities
students_tmp = get_data('data/priorities.json')

# read special cases
specials = get_data('data/specials.json')

N_blocks = len(blocks) # number of blocks
N_students = len(students_tmp) # number of students
N_tries = 1000 # number of tries for optimisation loop

# initialise blocks
init_blocks(blocks)

# initialise student data
students = init_students(students_tmp, blocks)

# calculate factors for modules
calc_factors(blocks)

# calculate weights for students
calc_weights(students, N_blocks)

# optimisation
best_blocks = []

for block_nr in range(1, N_blocks+1):
    # get data for first try
    best_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
    block_penalty = get_block_penalty(best_block, block_nr, students)
    students_penalty = get_students_penalty(students, block_nr)
    penalty = block_penalty + students_penalty

    for i in range(N_tries):
        # get data for next try
        try_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
        block_penalty_try = get_block_penalty(try_block, block_nr, students)
        students_penalty_try = get_students_penalty(students, block_nr)
        penalty_try = block_penalty_try + students_penalty_try

        # pick best of new try and former best try    
        if penalty_try < penalty:
            best_block = try_block
            block_penalty = block_penalty_try
            students_penalty = students_penalty_try
            penalty = penalty_try

    # update student selections       
    update_students(best_block, block_nr, students)

    best_blocks.append(best_block)


In [4]:
block_nr = 1
details = True

list_results(best_blocks, block_nr, students, details)

block 1: p1: 59.0%, p2: 32.0%, p3: 5.6%, p0: 3.4%
Zeix: 6/20, p1: 16.7% (10.0%), p2: 83.3% (140.0%), p3: 0.0% (120.0%), p0: 0.0%
SCS Supercomputing Systems: 14/20, p1: 78.6% (80.0%), p2: 21.4% (100.0%), p3: 0.0% (110.0%), p0: 0.0%
Kyburz: 20/20, p1: 75.0% (105.0%), p2: 20.0% (185.0%), p3: 0.0% (160.0%), p0: 5.0%
Siemens Mobility AG: 20/20, p1: 75.0% (100.0%), p2: 15.0% (80.0%), p3: 5.0% (160.0%), p0: 5.0%
Varian: 16/20, p1: 37.5% (40.0%), p2: 50.0% (260.0%), p3: 12.5% (130.0%), p0: 0.0%
MAN Energy Solutions: 2/20, p1: 50.0% (20.0%), p2: 0.0% (85.0%), p3: 50.0% (190.0%), p0: 0.0%
Universitätsspital Zürich: 20/20, p1: 80.0% (240.0%), p2: 15.0% (230.0%), p3: 5.0% (120.0%), p0: 0.0%
Acht Grad Ost: 10/20, p1: 60.0% (35.0%), p2: 30.0% (90.0%), p3: 10.0% (155.0%), p0: 0.0%
Weidmann: 12/20, p1: 25.0% (25.0%), p2: 50.0% (210.0%), p3: 16.7% (140.0%), p0: 8.3%
True Wealth: 20/20, p1: 80.0% (115.0%), p2: 20.0% (155.0%), p3: 0.0% (105.0%), p0: 0.0%
PartnerRe: 7/20, p1: 42.9% (20.0%), p2: 42.9% (90.

In [6]:
module_nr = 3

for module in best_blocks[module_nr]:
    print(module, get_classes_for_module(best_blocks[module_nr], module, students))

Aktuelle Forschung am CERN (UZH) ['3b', '3c', '3d', '3e', '3f', '3g', '3h']
Hyperwürfel (UZH) ['3a', '3b', '3d', '3e', '3f', '3g', '3h']
Krebsforschung (UZH) ['3a', '3b', '3c', '3d', '3e', '3f', '3g']
Informatik (ETH) ['3a', '3b', '3c', '3f', '3g', '3h']
Informationstechnologie und Elektrotechnik (ETH) ['3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h']
Maschineningenieurwissenschaften (ETH) ['3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h']
Mathematik (ETH) ['3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h']


In [7]:
get_incomplete_students(students, 5)

[]